In [27]:
# -*- coding: utf-8 -*-
%matplotlib inline

import pandas as pd
import numpy as np
import math
import matplotlib as mplstyle
import matplotlib.pyplot as plt
import locale
import arrow
from locale import atof
from dateutil import parser
from datetime import datetime

### Functions

In [28]:
# Remove punctuation
def removeFormat(item):
    try:
        tempVal = item.replace(',','')
        val = int(tempVal)
        return val
    except:
        return float('nan')
    
# Simple counter
def counter(item):
    return 1

### Imports

In [29]:
df_vacc = pd.read_csv('../data/imports/dtp1.csv')
df_pop = pd.read_csv('../data/imports/population-under14.csv')
df_gdp = pd.read_csv('../data/imports/GDP2016.csv')
df_ppp = pd.read_csv('../data/imports/PPP.csv')
df_pppMeta = pd.read_csv('../data/imports/PPP_meta.csv')
df_graphicExample = pd.read_csv('../data/imports/graphic-copy-sample.csv')
df_completeCountryList = pd.read_csv('../data/exports/complete-country-list.csv')
df_filteredCountryList = pd.read_csv('../data/exports/filtered-country-list.csv')
df_pop_cleaned = pd.read_csv('../data/exports/cleaning/population-under14-cleaned.csv')

### Analysis

In [30]:
# Merge vaccine data and GDP
df_vacGdpMerge = df_vacc.merge(df_gdp, on='iso3', how='outer')
df_vacGdpMerge.to_csv('../data/exports/df_vacGdpMerge.csv')

In [31]:
# Prep population for merge 
df_pop_cleaned = df_pop_cleaned.rename(columns = {'Region, country or area' : 'country'})

# Merge vacGdp and population THIS IS PROBLEM
df_vacGdpPopMerge = df_vacGdpMerge.merge(df_pop_cleaned, on='country', how='outer')


In [32]:
# Clean merge
df_vacGdpPopMergeClean = df_vacGdpPopMerge.drop(['vaccine', 'economy', ' Male ', ' Female '], axis=1)
df_vacGdpPopMergeClean = df_vacGdpPopMergeClean.rename(columns = {'millions of US dollars':'GDP',' 0-14 (percentage) ':'Pop % under 14','Number under age 14':'Pop # under 14'})
df_vacGdpPopMergeClean['Pop # under 14'] = df_vacGdpPopMergeClean['Pop # under 14'].apply(removeFormat)

In [33]:
# Columns for condesnced df
cols = ['unicef_region','iso3','country','GDP','2016','1996','1986','Pop % under 14','Pop # under 14']

# Apply columns
df_vacGdpPopMergeCond = df_vacGdpPopMergeClean[cols]
df_vacGdpPopMergeCond = df_vacGdpPopMergeCond[np.isfinite(df_vacGdpPopMergeCond['2016'])]
df_vacGdpPopMergeCond.sort_values('GDP')
df_vacGdpPopMergeCond.to_csv('../data/exports/df_vacGdpPopMergeCond.csv')

In [34]:
# Prep to merge PPP and PPP Meta

# Raname columns for merge
df_ppp = df_ppp.rename(columns = {'Country Code':'iso3','2016':'PPP 2016'})
df_pppMeta = df_pppMeta.rename(columns = {'Country Code':'iso3'})

# Define columns to keep
pppCols = ['iso3','PPP 2016']
pppMetaCols = ['iso3','IncomeGroup']

# Apply columns to dataframes
df_pppCond = df_ppp[pppCols]
df_pppMetaCond = df_pppMeta[pppMetaCols]

# Merge PPP files together
df_pppMerge = df_pppCond.merge(df_pppMetaCond,on='iso3',how='outer')

In [35]:
df_pppMerge

,iso3,PPP 2016,IncomeGroup
0,ABW,NaN,High income
1,AFG,1876.544682,Low income
2,AGO,6499.067104,Lower middle income
3,ALB,11928.536560,Upper middle income
4,AND,NaN,High income
5,ARB,16697.596840,NaN
6,ARE,72418.610860,High income
7,ARG,19934.372490,Upper middle income
8,ARM,8817.953996,Lower middle income
9,ASM,NaN,Upper middle income


In [42]:
# Merge df_vacGdpPopMergeCond and df_pppMerge then rename columns
df_finalMerge = df_vacGdpPopMergeCond.merge(df_pppMerge, on='iso3', how='outer')
df_finalMergeClean = df_finalMerge.rename(columns = {'GDP':'GDP 2016','2016':'DTP1-16','1996':'DTP1-96','1986':'DTP1-86'})
# list(df_finalMerge)

# Filter for blanks
df_finalMergeClean = df_finalMergeClean[np.isfinite(df_finalMerge['2016'])]

# Reorder
list(df_finalMergeClean)
cols = ['unicef_region','iso3','country','IncomeGroup','GDP 2016','PPP 2016','Pop % under 14','Pop # under 14','DTP1-16','DTP1-96','DTP1-86']
df_finalMergeClean = df_finalMergeClean[cols]
# df_finalMergeClean.to_csv('../data/exports/df_finalMergeClean.csv')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 0 to 194
Data columns (total 11 columns):
unicef_region     195 non-null object
iso3              195 non-null object
country           195 non-null object
IncomeGroup       193 non-null object
GDP 2016          193 non-null object
PPP 2016          175 non-null float64
Pop % under 14    184 non-null float64
Pop # under 14    184 non-null float64
DTP1-16           195 non-null float64
DTP1-96           189 non-null float64
DTP1-86           159 non-null float64
dtypes: float64(6), object(5)
memory usage: 18.3+ KB


In [37]:
# df_finalMergeClean.sort_values('country').info()
df_finalMergeClean[df_finalMergeClean['country'] == 'France']

,unicef_region,iso3,country,IncomeGroup,GDP 2016,PPP 2016,Pop % under 14,Pop # under 14,DTP1-16,DTP1-96,DTP1-86
61,Western Europe,FRA,France,High income,"2,465,454",41466.26571,18.0,11696400.0,99.0,98.0,98.0


In [38]:
# Assign columns for export
cols = ['iso3','Pop # under 14','DTP1-16']

# Drop unnecessary columns
df_mapDetailsCond = df_finalMergeClean[cols]

# Rename 
df_mapDetailsCond = df_mapDetailsCond.rename(columns={'country':'name','Pop # under 14':'population'})

In [43]:
# Merge with graphic example then drop rows with empty DTP cells and unnecessary columns
df_completeCountryList = df_completeCountryList.rename(columns={'ISO_A3':'iso3','NAME':'name'})
# df_completeCountryList

df_graphicSpreadsheet = df_completeCountryList.merge(df_mapDetailsCond, on='iso3', how='outer')
df_graphicSpreadsheet
# df_graphicExport = df_graphicSpreadsheet.merge(df_graphicExample,on='name',how='outer')

# df_graphicExportCond = df_graphicSpreadsheet[np.isfinite(df_graphicSpreadsheet['DTP1-16'])]
# df_graphicExportFinal = df_graphicExport[np.isfinite(df_graphicExport['DTP1-16'])]
# df_graphicExportCond

,Unnamed: 0,iso3,ADM0_DIF,LEVEL,ADM0_A3,ADM0_A3_IS,GU_A3,ISO_N3,TYPE,GEOUNIT,ABBREV,name,NAME_LONG,population,DTP1-16
0,0.0,ABW,1.0,2.0,ABW,ABW,ABW,533.0,Country,Aruba,Aruba,Aruba,Aruba,NaN,NaN
1,1.0,-99,0.0,3.0,ATG,ATG,ACA,-99.0,Geo unit,Antigua,Ant.,Antigua,Antigua,NaN,NaN
2,2.0,-99,0.0,3.0,ATG,ATG,ACB,-99.0,Geo unit,Barbuda,Barb.,Barbuda,Barbuda,NaN,NaN
3,14.0,-99,1.0,2.0,ATC,AUS,ATC,36.0,Dependency,Ashmore and Cartier Islands,A.C.Is.,Ashmore and Cartier Is.,Ashmore and Cartier Islands,NaN,NaN
4,19.0,-99,0.0,3.0,BEL,BEL,BCR,-99.0,Geo unit,Brussels Capital Region,Brus.,Brussels,Brussels,NaN,NaN
5,23.0,-99,0.0,3.0,BEL,BEL,BFR,-99.0,Geo unit,Flemish Region,Flem.,Flemish,Flemish Region,NaN,NaN
6,26.0,-99,0.0,3.0,BIH,BIH,BHB,-99.0,Geo unit,Brcko District,B.D.,Brcko District,Brcko District,NaN,NaN
7,27.0,-99,0.0,3.0,BIH,BIH,BHF,-99.0,Geo unit,Bosnia and Herzegovina,B.H.,Fed. of Bos. & Herz.,Federation of Bosnia and Herzegovina,NaN,NaN
8,30.0,-99,0.0,3.0,BIH,BIH,BIS,-99.0,Geo unit,Republic Srpska,Srp.,Rep. Srpska,Republic Srpska,NaN,NaN
9,31.0,-99,0.0,2.0,BJN,-99,BJN,-99.0,Indeterminate,Bajo Nuevo Bank (Petrel Is.),NaN,Bajo Nuevo Bank (Petrel Is.),Bajo Nuevo Bank (Petrel Islands),NaN,NaN


In [40]:
# Export
df_graphicExportFinal.to_csv('../data/exports/df_graphicExportFinal.csv')

NameError: name 'df_graphicExportFinal' is not defined